# Zadanie 7 (7 pkt)
Celem zadania jest zaimplementowanie dwóch wersji naiwnego klasyfikatora Bayesa.
* W pierwszej wersji należy dokonać dyskretyzacji danych - przedział wartości każdego atrybutu dzielimy na cztery równe przedziały i każdej ciągłej wartości atrybutu przypisujemy wartość dyskretną wynikająca z przynależności do danego przedziału.
* W drugiej wersji wartości likelihood wyliczamy z rozkładów normalnych o średnich i odchyleniach standardowych wynikających z wartości atrybutów.

Trening i test należy przeprowadzić dla zbioru Iris, tak jak w przypadku zadania z drzewem klasyfikacyjnym. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania klasyfikatorów dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Dyskretyzacja danych - **0.5 pkt**
* Implementacja funkcji rozkładu normalnego o zadanej średniej i odchyleniu standardowym. - **0.5 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych dyskretnych. - **2.0 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych ciągłych. - **2.5 pkt**
* Przeprowadzenie eksperymentów, wnioski i sposób ich prezentacji. - **1.5 pkt**

In [66]:
from collections import Counter
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


class NaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def build_classifier(self, train_features, train_classes):
        self.priors = Counter(train_classes)
        self.likelihoods = np.zeros(
            shape=(len(self.priors), train_features.shape[1], 4)
        )
        for features, result_class in zip(train_features, train_classes):
            for i, feature in enumerate(features):
                self.likelihoods[result_class, i, feature] += 1
        total = self.priors.total()
        for key in self.priors.keys():
            key_occurances = self.priors[key]
            self.priors[key] /= total
            for i in range(train_features.shape[1]):
                for j in range(4):
                    self.likelihoods[key, i, j] = (self.likelihoods[key, i, j] + 1) / (
                        key_occurances + 4
                    )

    @staticmethod
    def data_discretization(data, intervals):
        discretize = (
            lambda x: 0
            if x < intervals[0]
            else 1
            if x < intervals[1]
            else 2
            if x < intervals[2]
            else 3
        )
        return [discretize(x) for x in data]
    
    def find_intervals(self, train_data):
        self.intervals = np.zeros(shape=(train_data.shape[1], 3))
        for i, features in enumerate(train_data.T):
            max_value = max(features)
            min_value = min(features)
            section_size = (max_value - min_value) / 4
            first_section_limit = min_value + section_size
            second_section_limit = first_section_limit + section_size
            third_section_limit = second_section_limit + section_size
            self.intervals[i] = np.array([first_section_limit, second_section_limit, third_section_limit])

    def predict(self, sample):
        max_probability = 0
        prediction = None
        for key in self.priors.keys():
            probability = self.priors[key]
            for i, feature in enumerate(sample):
                probability *= self.likelihoods[key, i, feature]
            if probability > max_probability:
                prediction = key
                max_probability = probability
        return prediction


class GaussianNaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def build_classifier(self, train_features, train_classes):
        self.priors = Counter(train_classes)
        self.likelihoods = np.zeros(
            shape=(len(self.priors), train_features.shape[1], 2)
        )
        total = self.priors.total()
        for key in self.priors.keys():
            self.priors[key] /= total
            indices = np.where(train_classes == key)
            for i, feature in enumerate(train_features[indices].T):
                mean = np.mean(feature)
                deviation = np.std(feature, ddof=1)
                self.likelihoods[key, i, 0] = mean
                self.likelihoods[key, i, 1] = deviation

    @staticmethod
    def normal_dist(x, mean, std):
        return (
            1
            / (std * np.sqrt(2 * np.sqrt(2 * np.pi)))
            * np.e ** (-0.5 * ((x - mean) / std) ** 2)
        )

    def predict(self, sample):
        max_probability = 0
        prediction = None
        for key in self.priors.keys():
            probability = self.priors[key]
            for i, feature in enumerate(sample):
                mean = self.likelihoods[key, i, 0]
                deviation = self.likelihoods[key, i, 1]
                probability *= GaussianNaiveBayes.normal_dist(feature, mean, deviation)
            if probability > max_probability:
                prediction = key
                max_probability = probability
        return prediction


In [76]:
iris = load_iris()

x = iris.data
y = iris.target

# x = np.array([NaiveBayes.data_discretization(r) for r in x.T]).T

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1, random_state=123
)

nb = NaiveBayes()
nb.find_intervals(x_train)
intervals = nb.intervals
x_train = np.array([NaiveBayes.data_discretization(features, intervals[i]) for i, features in enumerate(x_train.T)]).T
x_test = np.array([NaiveBayes.data_discretization(features, intervals[i]) for i, features in enumerate(x_test.T)]).T

good = 0
total = 0
nb = NaiveBayes()
nb.build_classifier(x_train, y_train)
for test_x, test_y in zip(x_test, y_test):
    prediction = nb.predict(test_x)
    print(f"Prediction: {prediction}, True class: {test_y}")
    if prediction == test_y:
        good += 1
    total += 1
print(f"Accuracy: {good/total:.3f}")

Prediction: 1, True class: 1
Prediction: 2, True class: 2
Prediction: 2, True class: 2
Prediction: 1, True class: 1
Prediction: 0, True class: 0
Prediction: 1, True class: 2
Prediction: 1, True class: 1
Prediction: 0, True class: 0
Prediction: 0, True class: 0
Prediction: 1, True class: 1
Prediction: 2, True class: 2
Prediction: 0, True class: 0
Prediction: 1, True class: 1
Prediction: 2, True class: 2
Prediction: 2, True class: 2
Accuracy: 0.933


In [68]:
iris = load_iris()

x = iris.data
y = iris.target


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)
good = 0
total = 0
nb = GaussianNaiveBayes()
nb.build_classifier(x_train, y_train)
for test_x, test_y in zip(x_test, y_test):
    prediction = nb.predict(test_x)
    print(f"Prediction: {prediction}, True class: {test_y}")
    if prediction == test_y:
        good += 1
    total += 1
print(f"Accuracy: {good/total:.3f}")

Prediction: 1, True class: 1
Prediction: 2, True class: 2
Prediction: 2, True class: 2
Prediction: 1, True class: 1
Prediction: 0, True class: 0
Prediction: 2, True class: 2
Prediction: 1, True class: 1
Prediction: 0, True class: 0
Prediction: 0, True class: 0
Prediction: 1, True class: 1
Prediction: 2, True class: 2
Prediction: 0, True class: 0
Prediction: 1, True class: 1
Prediction: 2, True class: 2
Prediction: 2, True class: 2
Accuracy: 1.000


# Wnioski